In [ ]:
# Install dependencies
!pip install transformers torch termcolor pyfiglet

# Import dependencies
import re
import time
import random
import datetime
from transformers import pipeline
from termcolor import colored
import pyfiglet

# Display Title
title = pyfiglet.figlet_format("MedBot")
print(colored(title, "cyan"))

# Load Emotion and Sentiment Classifiers
print(colored("Loading models...", "yellow"))
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-go-emotion", top_k=None)
sentiment_classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Symptom mapping
symptom_variations = {
    "dizzy": "dizziness", "nauseous": "nausea", "nausea": "nausea", "dizziness": "dizziness",
    "fever": "fever", "headache": "headache", "cold": "cold", "sore throat": "sore throat",
    "throat pain": "sore throat", "throat irritation": "sore throat", "cancer": "cancer",
    "lung cancer": "lung cancer", "breast cancer": "breast cancer", "skin cancer": "skin cancer",
    "blood cancer": "blood cancer", "tuberculosis": "tuberculosis", "diabetes": "diabetes",
    "hypertension": "hypertension", "asthma": "asthma", "dengue": "dengue", "malaria": "malaria",
    "pneumonia": "pneumonia", "migraine": "migraine", "skin infection": "skin infection",
    "anemia": "anemia", "depression": "depression"
}

# Treatment and info
treatment_guidelines = {
    "eye pain": {"med": "Lubricant drops", "dur": "3-5 days", "note": "Avoid screen strain", "pain": "Mild", "cat": "Ophthalmology"},
    "fever": {"medication": "Paracetamol 500mg", "duration": "3-5 days", "note": "Take after meals.", "doctor_required": False, "pain": "Mild", "category": "Infection"},
    "dizziness": {"medication": "Meclizine", "duration": "2-3 days", "note": "Avoid driving.", "doctor_required": False, "pain": "Mild", "category": "Neurological"},
    "nausea": {"medication": "Ondansetron", "duration": "1-2 days", "note": "Every 8 hours.", "doctor_required": False, "pain": "Mild", "category": "Digestive"},
    "sore throat": {"medication": "Cefixime or Lozenges", "duration": "5 days", "note": "Cefixime if bacterial.", "doctor_required": False, "pain": "Mild", "category": "ENT"},
    "headache": {"medication": "Ibuprofen 400mg", "duration": "1-3 days", "note": "After meals.", "doctor_required": False, "pain": "Moderate", "category": "Neurological"},
    "cold": {"medication": "Cetirizine 10mg", "duration": "3-5 days", "note": "At night.", "doctor_required": False, "pain": "Mild", "category": "Respiratory"},
    "cancer": {"info": "Cancer refers to uncontrolled cell growth. Consult an oncologist.", "treatment": ["Surgery", "Chemotherapy", "Radiation"], "doctor_required": True, "pain": "Chronic", "category": "Oncology"},
    "lung cancer": {"info": "Lung cancer affects the lungs. Often caused by smoking.", "doctor_required": True, "pain": "Chronic", "category": "Oncology"},
    "breast cancer": {"info": "Breast cancer occurs in breast cells. Regular screening is key.", "doctor_required": True, "pain": "Chronic", "category": "Oncology"},
    "skin cancer": {"info": "Skin cancer develops on sun-exposed areas. Early detection helps.", "doctor_required": True, "pain": "Mild", "category": "Oncology"},
    "blood cancer": {"info": "Blood cancer affects blood-forming tissues.", "doctor_required": True, "pain": "Moderate", "category": "Oncology"},
    "diabetes": {"medication": "Insulin or Metformin", "duration": "Ongoing", "note": "Diet control needed.", "doctor_required": True, "pain": "None", "category": "Metabolic"},
    "hypertension": {"medication": "Amlodipine or Losartan", "duration": "Ongoing", "note": "Check BP regularly.", "doctor_required": True, "pain": "None", "category": "Cardiovascular"},
    "tuberculosis": {"info": "TB needs long antibiotic therapy.", "doctor_required": True, "pain": "Mild", "category": "Infectious"},
    "asthma": {"medication": "Albuterol inhaler", "duration": "As needed", "note": "Carry inhaler.", "doctor_required": True, "pain": "None", "category": "Respiratory"},
    "dengue": {"medication": "Rest & fluids", "duration": "7-10 days", "note": "Hydration critical.", "doctor_required": True, "pain": "Severe", "category": "Infectious"},
    "malaria": {"medication": "Artemisinin", "duration": "7-14 days", "note": "Prevent mosquito bites.", "doctor_required": True, "pain": "Severe", "category": "Infectious"},
    "pneumonia": {"medication": "Antibiotics & rest", "duration": "7-10 days", "note": "Seek care if severe.", "doctor_required": True, "pain": "Moderate", "category": "Respiratory"},
    "migraine": {"medication": "Sumatriptan", "duration": "Ongoing", "note": "Manage with rest.", "doctor_required": False, "pain": "Severe", "category": "Neurological"},
    "skin infection": {"medication": "Antifungals or antibiotics", "duration": "7 days", "note": "Apply topically.", "doctor_required": False, "pain": "Mild", "category": "Dermatology"},
    "anemia": {"medication": "Iron supplements", "duration": "Ongoing", "note": "Eat iron-rich foods.", "doctor_required": False, "pain": "Fatigue", "category": "Hematology"},
    "depression": {"treatment": ["Therapy", "Medication"], "note": "Consult psychiatrist.", "doctor_required": True, "pain": "Emotional", "category": "Mental Health"}
}

# Diet Tips Dictionary
diet_tips = {
    "cancer": {
        "diet": ["Lean proteins", "Fruits and vegetables", "Whole grains", "Healthy fats"],
        "avoid": ["Processed meats", "Excess sugar", "Alcohol", "High-salt foods"],
        "note": "Eat small frequent meals. Stay hydrated."
    },
    "diabetes": {
        "diet": ["Whole grains", "Leafy greens", "Low-GI fruits", "Lean protein"],
        "avoid": ["Sugary foods", "Refined carbs", "Trans fats"],
        "note": "Eat balanced meals and monitor blood sugar."
    },
    "eye pain": {"eat": ["Carrots", "Spinach"], "avoid": ["Excess sugar"], "note": "Rest eyes regularly"},

    "hypertension": {
        "diet": ["Low-sodium foods", "Fruits", "Vegetables", "Omega-3s"],
        "avoid": ["Salt", "Caffeine", "Processed snacks"],
        "note": "Exercise regularly and reduce stress."
    },
    "anemia": {
        "diet": ["Iron-rich foods (spinach, red meat)", "Vitamin C-rich foods"],
        "avoid": ["Tea/coffee with meals"],
        "note": "Take iron supplements if prescribed."
    },
    "asthma": {
        "diet": ["Vitamin D foods", "Fruits and veggies", "Omega-3s"],
        "avoid": ["Processed foods", "Allergens (check individual triggers)"],
        "note": "Avoid cold foods if triggering."
    }
}

# Emotion & Sentiment Detection
def detect_emotion(text):
    emotions = emotion_classifier(text)[0]
    for e in emotions:
        if e['label'] in ['fear', 'anxiety', 'sadness'] and e['score'] > 0.7:
            return True
    return False

def detect_sentiment(text):
    result = sentiment_classifier(text)[0]
    return result['label'], result['score']

# Core Medical Advice Function
def get_medical_advice(text, emotion_detected=False, sentiment_label=None):
    input_text = text.lower()
    responses = []

    for variant, details in treatment_guidelines.items():
        if variant in input_text:
            response = f"\n💠 General Info: {details.get('info', 'Symptoms detected for ' + variant)}"
            if "medication" in details:
                response += f"\n💊 Medication: {details['medication']}"
                response += f"\n📅 Duration: {details['duration']}"
                response += f"\n📝 Note: {details['note']}"
            if "treatment" in details:
                response += f"\n💉 Treatment Options: {', '.join(details['treatment'])}"
            response += f"\n🤕 Pain Type: {details.get('pain', 'Varies')}"
            response += f"\n🧬 Disease Category: {details.get('category', 'General')}"
            if details.get("doctor_required"):
                response += f"\n👨‍⚕ Doctor Consultation: *Required*"
            else:
                response += f"\n👨‍⚕ Doctor Consultation: Optional"
            response += f"\n📆 Date: {datetime.datetime.today().strftime('%Y-%m-%d')}"
            if sentiment_label:
                response += f"\n💗 Sentiment Detected: *{sentiment_label.upper()}*"
            if variant in diet_tips:
                diet = diet_tips[variant]
                response += f"\n🥗 Recommended Diet: {', '.join(diet['diet'])}"
                response += f"\n🚫 Avoid: {', '.join(diet['avoid'])}"
                response += f"\n📝 Diet Note: {diet['note']}"
            responses.append(response)

    if not responses:
        if sentiment_label == "POSITIVE":
            responses.append("I'm glad you're feeling okay! Let me know if you need any advice.")
        elif emotion_detected or sentiment_label == "NEGATIVE":
            responses.append("You seem concerned. Please rest and consult a doctor if you're unwell.")
        else:
            responses.append("Tell me more about how you're feeling!")

    return "\n".join(responses)

# Start Chat Loop
print(colored("MedBot is online. Type 'exit' to quit.\n", "green"))

while True:
    user_input = input(colored("You: ", "blue"))
    if user_input.lower() == 'exit':
        print(colored("MedBot: Take care! Stay safe. 👋", "green"))
        break

    emotion_flag = detect_emotion(user_input)
    sentiment_label, sentiment_score = detect_sentiment(user_input)

    response = get_medical_advice(user_input, emotion_flag, sentiment_label)
    print(colored(response, "yellow"))  # <-- Moved this inside the loop


 __  __          _ ____        _   
|  \/  | ___  __| | __ )  ___ | |_ 
| |\/| |/ _ \/ _` |  _ \ / _ \| __|
| |  | |  __/ (_| | |_) | (_) | |_ 
|_|  |_|\___|\__,_|____/ \___/ \__|
                                   

Loading models...


Device set to use cpu
Device set to use cpu


MedBot is online. Type 'exit' to quit.

You: cancer

💠 General Info: Cancer refers to uncontrolled cell growth. Consult an oncologist.
💉 Treatment Options: Surgery, Chemotherapy, Radiation
🤕 Pain Type: Chronic
🧬 Disease Category: Oncology
👨‍⚕ Doctor Consultation: *Required*
📆 Date: 2025-05-25
💗 Sentiment Detected: *NEGATIVE*
🥗 Recommended Diet: Lean proteins, Fruits and vegetables, Whole grains, Healthy fats
🚫 Avoid: Processed meats, Excess sugar, Alcohol, High-salt foods
📝 Diet Note: Eat small frequent meals. Stay hydrated.
You: hi
I'm glad you're feeling okay! Let me know if you need any advice.
You: hello
I'm glad you're feeling okay! Let me know if you need any advice.
You: cold

💠 General Info: Symptoms detected for cold
💊 Medication: Cetirizine 10mg
📅 Duration: 3-5 days
📝 Note: At night.
🤕 Pain Type: Mild
🧬 Disease Category: Respiratory
👨‍⚕ Doctor Consultation: Optional
📆 Date: 2025-05-25
💗 Sentiment Detected: *NEGATIVE*
You: i am suffering from head ache
You seem concerned. Plea